# Run Predict Demo

In [2]:
from pathlib import Path
import torch
import librosa
import CNN1D_MDPI_5s
from get_transform import get_transform

In [31]:
x, sr = librosa.load('../sound_tmp/php1AB2.mp3', sr=44100, mono=True, duration=5)
# audio, sr = librosa.load(Path('..', 'sound_tmp', 'php1AB2.mp3'), sr=44100)

C:\Users\zhiren\AppData\Local\Temp\ipykernel_18312\2583134934.py:1: UserWarning: PySoundFile failed. Trying audioread instead.
  x, sr = librosa.load('../sound_tmp/php1AB2.mp3', sr=44100, mono=True, duration=5)
c:\Users\zhiren\Envs\asp-azhmr\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [32]:
x.shape

(220500,)

## Preprocessing for fit the model

In [33]:
x_duration = librosa.get_duration(y=x, sr=sr)

In [34]:
x_duration

5.0

In [35]:
model_fit_duration = 5.0

In [36]:
x_duration > model_fit_duration

False

In [38]:
if x_duration < model_fit_duration:
    # 拉伸至5s
    stretch_rate = x_duration / model_fit_duration
    y = librosa.effects.time_stretch(y=x, rate=stretch_rate)
    

In [39]:
x_duration = librosa.get_duration(y=x, sr=sr)

In [40]:
x_duration

5.0

In [41]:
transform = get_transform()

In [42]:
x_processed = transform(x)

c:\Users\zhiren\Programs\aidd\scripts\get_transform.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(torch.from_numpy(y),


In [43]:
x_toDevice = x_processed.to('cuda')

## Model initialize

In [45]:
model = torch.load(Path('pth', 'best.pth'), map_location=torch.device('cpu'))

In [46]:
model.to('cuda')

CNN1D_MDPI_5(
  (conv1): Conv1d(128, 32, kernel_size=(32,), stride=(1,), padding=(1,))
  (conv2): Conv1d(32, 64, kernel_size=(18,), stride=(1,), padding=(1,))
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=512, out_features=4, bias=True)
  (linear2): Linear(in_features=4, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

In [47]:
model.eval()

CNN1D_MDPI_5(
  (conv1): Conv1d(128, 32, kernel_size=(32,), stride=(1,), padding=(1,))
  (conv2): Conv1d(32, 64, kernel_size=(18,), stride=(1,), padding=(1,))
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=512, out_features=4, bias=True)
  (linear2): Linear(in_features=4, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

In [48]:
y_pred = model(x_toDevice)

In [49]:
y_pred.argmax(1).cpu().numpy()[0]

0